In [34]:
import importlib
import logging

from tqdm import tqdm

from datetime import datetime, timedelta

from IPython.display import display_latex
import units.smuless as smu
importlib.reload(smu)
# logging config
smu.PumpLogger.setLevel(logging.DEBUG)
smu.SmLogger.setLevel(logging.DEBUG)
%config Application.logging_config = {'loggers': {'ct.sm': {'propagate': True, 'level': 'DEBUG', 'handlers': ['console']}}}

In [35]:
canist_ref = smu.GasMixture.pure('N2', 293.15, 1, 90*101325)
canist_ref

GasMixture(T=293.15, V=1, nus={'N2': 3743.416924714803})

In [192]:
class SmModel:
    he_i = None
    teg_pwr = 0#*u.J
    def __init__(self):
        self.core_room = smu.GasMixture(smu.T_MINIMAL, 15*smu.CELL_VOLUME, {})
        self.core_room._gr = 15
        self.smcore = smu.Supermatter(self.core_room) #
        # pipe networks with pipelines as base
        self.cooled   = smu.PipeNetwork({'cooled':   smu.GasMixture(0,   .070)})
        self.cooling  = smu.PipeNetwork({'cooling':  smu.GasMixture(0, 15.680)})
        self.warm     = smu.PipeNetwork({'warm':     smu.GasMixture(0,  1.890)})
        self.warm.add('filter1', 1)
        self.warm.add('filter2', 1)
        self.warm.add('nopump',   .2) # empty for waste or whatever
        self.scalding = smu.PipeNetwork({'scalding': smu.GasMixture(0, .315)})
        # connectors
        self.c_hot =  smu.PipeNetwork({'':canist_ref*1})
        self.c_cold = smu.PipeNetwork({'':canist_ref*1})
        # pumps
        self.po1 = smu.OutletInjector('PO1', P_max=30e3, target_V=0.700) # Outlet Injector
        self.pi1 = smu.VentPump('PI1', V_out=0.700, ex_P = 100e3) # , V_in=0*u.L, V_out=0.2*u.m**3) # Vent Pump
        self.hp1 = smu.Pump('HP1', P_max=45e3)
        self.pc_hot =  smu.Pump('P[C]hot') # C2-P2 on pic
        self.pc_cold = smu.Pump('P[C]cold') # C5-P6 on pic
        # TEG and HE
        self.t_hot =  smu.Turbine('T(hot)1')
        self.t_cold = smu.Turbine('T(cold)2')
        self.teg = smu.TEG(self.t_hot, self.t_cold)
        self.he = smu.HeatExchanger(191)
        # -- connecting devices --
        self.pc_hot.connect( self.c_hot,     self.warm)
        self.pc_cold.connect(self.c_cold,    self.cooling)
        self.hp1.connect(    self.cooling,   self.cooled)
        self.pi1.connect(    self.core_room, self.scalding)
        self.po1.connect(    self.warm,      self.core_room)
        self.t_hot.connect(  self.scalding,  self.warm)
        self.t_cold.connect( self.cooled,    self.cooling)

    def energize_core(self, n_pulses: int):
        self.smcore.eer = smu.POWERED_EMITTER_DMG*n_pulses/smu.SM_EER_EMIT

    def pump_cold(self):
        if self.he_i is None: self.he_i = 0
        if self.he_i > 1: 
            self.he.cool(self.cooling.gases['cooling'])
        [x.equalize() for x in [self.c_cold, self.cooling]]
        self.pc_cold.pump()
        self.he_i += 1

    def pump_hot_teg(self):
        pwr_lost = [x.pump() for x in [self.pc_hot, self.pi1, self.po1]]
        self.teg_pwr = self.teg.process()
        [x.equalize() for x in [self.c_hot, self.warm, self.scalding]]

    def full_tick(self):
        self.pump_cold()
        self.hp1.pump()
        self.pump_hot_teg()
        self.smcore.process()
        [x.equalize() for x in [self.c_hot, self.warm, self.scalding, self.c_cold, self.cooling, self.cooled]]
        self.core_room.react()

    @property
    def reftime(self):
        # t0 = time(0,0,0)
        if self.he_i is None: return timedelta(seconds=0)
        dt = timedelta(seconds=self.he_i*2)
        return (dt)

    def _repr_latex_(self) -> str:
        return '\n\n'.join([
            f'Ticks spent: {self.he_i} ; server time: {self.reftime}; ',
            f'Last power: {self.teg_pwr/1e3:.2f} kW',
            self.smcore._repr_latex_(),
            # 'C[hot]: '  + self.c_hot._repr_latex_()[25:],
            # 'C[cold]: ' + self.c_cold._repr_latex_()[25:],
            'scalding: '+ self.scalding._repr_latex_()[25:],
            'warm: ' +    self.warm._repr_latex_()[25:],
            'cooling: ' + self.cooling._repr_latex_()[25:],
            'cooled: ' +  self.cooled._repr_latex_()[25:],
        ])


In [193]:
def try_run(n: int, queue_hot: list[smu.GasMixture], queue_cold: list[smu.GasMixture], dbg=True) -> tuple[SmModel,tuple]:
    m = SmModel()
    if len(queue_hot) and 'N2' not in queue_hot[0].nus:
        # replacing 
        chot = queue_hot.pop(0)
        m.c_hot.gases[''] = chot
        m.c_hot.equalize()
    if len(queue_cold) and 'N2' not in queue_cold[0].nus:
        # replacing 
        ccold = queue_cold.pop(0)
        m.c_cold.gases[''] = ccold
        m.c_cold.equalize()
    while len(queue_hot) + len(queue_cold) > 0 and m.c_cold.gases[''].p + m.c_hot.gases[''].p > 1000 :
        m.pump_cold()
        m.pump_hot_teg()
        if len(queue_cold) and m.c_cold.gases[''].p < 1000:
            m.c_cold.gases[''] = queue_cold.pop(0)
        if len(queue_hot) and m.c_hot.gases[''].p < 1000:
            m.c_hot.gases[''] = queue_hot.pop(0)
    if dbg:
        print(f'Pumping ended by {m.reftime}')
    m.energize_core(n)
    delam = False
    T0 = m.core_room.T - 5
    for i in range(10800):
        m.full_tick()
        if m.core_room.T > 5000:
            if dbg: print('delam')
            delam = True
            break
        if i % 100 == 0:
            if m.core_room.T < T0:
                break
            T0 = m.core_room.T
    if i==10800 and dbg:
        print('too long')
    if dbg:
        print(f'{m.teg_pwr/1e3:.2f} kW at {m.reftime}')
        display_latex(m.smcore)
    return m, (delam, )

In [194]:
m = SmModel()
m.c_cold.gases[''].p

7599375.0

In [203]:
best = None
for ncold in range(5):
    best = None
    for nhot in range(5):
        n0 = 4 if best is None else best[0]-1
        best = None
        for n in range(n0,60):
            m, (delam, ) = try_run(n, [canist_ref*i for i in [1]*ncold], [canist_ref*i for i in [1]*nhot], False)
            if delam: break
            best = (n, m.core_room.T, m.teg_pwr, m.reftime)
        n, T, P, t = best
        print(f'{ncold+1=} {nhot+1=} best: {n=: 2}, T = {T: 6.2f} K, power {P/1e3: 5.2f} kW, {t} to stable-ish')

ncold+1=1 nhot+1=1 best: n= 4, T =  4412.16 K, power  513.15 kW, 1:01:50 to stable-ish
ncold+1=1 nhot+1=2 best: n= 5, T =  4964.83 K, power  635.64 kW, 0:56:46 to stable-ish
ncold+1=1 nhot+1=3 best: n= 5, T =  4765.49 K, power  636.25 kW, 0:58:36 to stable-ish
ncold+1=1 nhot+1=4 best: n= 5, T =  4656.45 K, power  635.54 kW, 0:57:14 to stable-ish
ncold+1=1 nhot+1=5 best: n= 5, T =  4579.10 K, power  637.02 kW, 0:59:26 to stable-ish
ncold+1=2 nhot+1=1 best: n= 7, T =  4854.87 K, power  882.21 kW, 1:20:04 to stable-ish
ncold+1=2 nhot+1=2 best: n= 8, T =  4732.55 K, power  979.95 kW, 1:10:12 to stable-ish
ncold+1=2 nhot+1=3 best: n= 9, T =  4901.27 K, power  1084.32 kW, 1:05:16 to stable-ish
ncold+1=2 nhot+1=4 best: n= 9, T =  4697.61 K, power  1080.60 kW, 1:03:54 to stable-ish
ncold+1=2 nhot+1=5 best: n= 9, T =  4556.90 K, power  1079.03 kW, 1:06:06 to stable-ish
ncold+1=3 nhot+1=1 best: n= 9, T =  4992.15 K, power  1106.73 kW, 1:41:46 to stable-ish
ncold+1=3 nhot+1=2 best: n= 11, T =  49

In [197]:
try_run(4, [canist_ref*1], [])[0]

Pumping ended by 0:03:22
520.56 kW at 1:36:44


In [49]:
canist_he = canist_ref*.5
canist_he.nus = {'He': canist_ref.nus['N2']}

In [146]:
m = SmModel()
# m.c_cold.gases[''] = canist_he*1
# m.c_cold.equalize()
# m.c_hot.gases[''] = canist_he*1
# m.c_hot.equalize()
for i in tqdm(range(5)):
    for _ in range(60):
        m.pump_cold()
        m.pump_hot_teg()
    #reload canister twice
    if i < 5:
        m.c_hot.gases['']  = canist_ref*1
        m.c_cold.gases[''] = canist_ref*1
    # print('Cooling [HE]')
    # display_latex(m.cooling.gases['cooling'].p.to(u.kPa).round(2))
    # print('Warm')
    # display_latex(m.warm.gases['warm'].p.to(u.kPa).round(2))
    # print('Core room')
    display_latex(m.smcore)

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:00<00:02,  1.70it/s]

 40%|████      | 2/5 [00:01<00:01,  1.75it/s]

 60%|██████    | 3/5 [00:01<00:01,  1.76it/s]

 80%|████████  | 4/5 [00:02<00:00,  1.77it/s]

100%|██████████| 5/5 [00:02<00:00,  1.77it/s]


In [147]:
m.energize_core(16)

In [148]:
T0 = m.smcore.room.T * 1 - 5
for i in tqdm(range(5000)):
    m.full_tick()
    if i % 100 == 0:
        if m.smcore.room.T < T0:
            print('stable-ish?')
            break
        T0 = m.smcore.room.T * 1
        if T0>5000:
            print('delam')
            break

  1%|          | 28/5000 [00:00<01:11, 69.52it/s]

 58%|█████▊    | 2900/5000 [00:35<00:25, 81.22it/s]

stable-ish?


In [163]:
10800*2/60/60

6.0